In [1]:
#%load_ext autoreload
#%autoreload 2

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('../modules'))

In [3]:
import pandas as pd
import numpy as np
import folium

import own.data.countries as Countries
import own.data.ecdc as ECDC

In [4]:
import own.funny as funny

# Data prep

In [5]:
countriesGeojson = Countries.geometries_110m_json()

In [6]:
deaths = ECDC.deaths().unstack()

# Remove NaN by setting the to null (since NaN corrupts color mapping)
deaths.fillna(0.0, inplace=True)

# Make columns cumulative sum columns
deaths = deaths.cumsum(skipna=True)

first_death_per_country = ECDC.first_death_date_per_country()
population = ECDC.population()

# Misc

In [7]:
import branca.colormap as cm

cmap = cm.LinearColormap(
    colors=['green','lightgreen','yellow','red','brown'],
    index=[0, 100, 1500, 30000, 50000],
    vmin=0,
#    vmax=max_color,
    vmax=100000,
)

if funny.is_display():
    cmap

# Choropleth

In [8]:
geo_deaths = deaths.loc[deaths.index.max()]

geo = countriesGeojson

for feature in geo['features']:
    
    country_code = feature['id']
    
# Inlcude total number of deaths in geojson
    if country_code in geo_deaths.index.values:
        value = geo_deaths[country_code]
    else:
        value = -1
    feature['properties']['deaths'] = value

# Include date of first death in geojson
    if (country_code in first_death_per_country.index):
        date = first_death_per_country.loc[country_code]['date']
        if (not pd.isnull(date)):
            value = date.strftime('%Y-%m-%d')
        else:
            value = None
    else:
        value = None
    feature['properties']['first_death'] = value
    
# Include population in geojson
    if (country_code in population.index):
        count = population.loc[country_code]['population']
        if (not np.NaN == count):
            value = count
        else:
            value = None
    else:
        value = None
    feature['properties']['population'] = value

In [9]:
def generate_choropleth(source):

    map = folium.Map(location=[25, 0], tiles='cartodbpositron', zoom_start=2)

    geojson = folium.GeoJson(
        source,
        style_function = lambda feature: {
            'fillColor': cmap(feature['properties']['deaths']),
            'fillOpacity': 0.80,
            'color' : 'white',
            'weight' : 1,
            'dashArray' : '5, 5'
            }
        ).add_to(map)

    geojson.add_child(
        folium.features.GeoJsonTooltip(
            [
                'name_en',
                'deaths',
                'first_death',
                'population',
                #'iso3166_a3',
            ],
            aliases = [
                'Country',
                'Total deaths',
                'First death data',
                'Population (2018)',
                #'country code',
            ],
            labels=True,
            #style='background-color: #0099cc;',
            opacity=0.75,
            className='x',
            attribution='...'
        )
    )

    cmap.add_to(map)

    return map

map = generate_choropleth(countriesGeojson)

if funny.is_save_for_later():
    funny.save_for_later('ecdc-choropleth', map)
    
if funny.is_save_to_public():
    map.save('../docs/plots/ecdc-choropleth.html')

if funny.is_display():
    display(map)

# TimeSliderChoropleth

In [10]:
# Unly use a subset from a certain date
deaths_epoch = deaths.loc['2020-03-01':]
# Translate datetime index into unix epoch time
deaths_epoch.index = deaths_epoch.index.astype(np.int64) // 10 ** 9

styledata = {}

# Iterate over all countries
for country in deaths_epoch.keys():
    # Create dateframe per country
    styledata[country] = pd.DataFrame(
        {
            'color': deaths_epoch[country],
            'opacity': deaths_epoch[country],
            'value': deaths_epoch[country], # This is actually not used
        }
    )

In [11]:
max_color, min_color, max_opacity, min_opacity = 0, 0, 0, 0

for country, data in styledata.items():
    max_color = max(max_color, data['color'].max())
    min_color = min(max_color, data['color'].min())
    max_opacity = max(max_color, data['opacity'].max())
    max_opacity = min(max_color, data['opacity'].max())


def norm(x):
    return (x - x.min()) / (x.max() - x.min())     + 0.10
#    return 0.75

for country, data in styledata.items():
    data['color'] = data['color'].apply(cmap)
    data['opacity'] = norm(data['opacity'])

In [12]:
styledict = {
    str(country): data.drop('value', axis=1).to_dict(orient='index') for
        country, data in styledata.items()
}

In [13]:
from folium.plugins import TimeSliderChoropleth

def generate_timeslider_choropleth():
    map = folium.Map([0, 0], zoom_start=2)

    g = TimeSliderChoropleth(
        countriesGeojson,
        styledict=styledict,
        name="dummy_name_for_d3"
    ).add_to(map)

    cmap.add_to(map)

    return map

map = generate_timeslider_choropleth()

if funny.is_save_for_later():
    funny.save_for_later('ecdc-choropleth-timed', map)
    
if funny.is_save_to_public():
    map.save('../docs/plots/ecdc-choropleth-timed.html')

if funny.is_display():
    display(map)